# 02. Sector Info Update

**목적**: `yfinance`를 활용하여 471개 미확인 기업(Others)의 Sector/Industry 정보를 보완함.

---

In [3]:
# Library Imports
import pandas as pd
import yfinance as yf
import time
from pathlib import Path
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# 경로 설정
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / 'Data_set'

df_daily = pd.read_csv(DATA_DIR / 'stock_daily_master.csv')
print("Loaded from CSV")

print(f"Loaded Data: {df_daily.shape}")

Loaded from CSV
Loaded Data: (602962, 27)
미확인 기업 수: 0
[]


## 1. 핵심 개념 정리: Ticker란?
이 코드에서 사용되는 `Ticker`라는 용어는 두 가지 의미를 가진다

1.  **주식 용어로서의 Ticker**: 기업을 식별하는 고유한 영문 약어 (예: 애플=`AAPL`, 삼성전자=`005930.KS`)
2.  **라이브러리 기능으로서의 `yf.Tickers`**: `yfinance` 라이브러리에서 **여러 종목을 한 번에 묶어서 처리**해주는 객체. "종목 바구니" 역할을 합니다.

In [ ]:
# 3. 정보 수집 함수 정의
def get_company_info(ticker_list):
    info_dict = {}
    
    chunk_size = 50
    chunks = [ticker_list[i:i + chunk_size] for i in range(0, len(ticker_list), chunk_size)]
        
    for chunk in tqdm(chunks, desc="Processing Chunks"):
        tickers_str = ' '.join(chunk)
        try:
            tickers = yf.Tickers(tickers_str)
            
            for symbol in chunk:
                try:
                    # 개별 ticker 정보 접근
                    info = tickers.tickers[symbol].info
                    
                    sector = info.get('sector', 'Unknown')
                    industry = info.get('industry', 'Unknown')
                    
                    # Unknown이면 개별 호출 재시도
                    if sector == 'Unknown':
                        single = yf.Ticker(symbol)
                        sector = single.info.get('sector', 'Unknown')
                        industry = single.info.get('industry', 'Unknown')

                    info_dict[symbol] = {
                        'Sector': sector,
                        'Industry': industry
                    }
                    
                except Exception as e:
                    # print(f"Error fetching {symbol}: {e}")
                    info_dict[symbol] = {'Sector': 'Error', 'Industry': 'Error'}
                
                # API 호출 제한 방지
                time.sleep(0.05)
                
        except Exception as e:
            print(f"Chunk Error: {e}")
            for symbol in chunk:
                 info_dict[symbol] = {'Sector': 'Error', 'Industry': 'Error'}
            
    return info_dict

1.  **청킹**: 500개 종목을 50개씩 나누어 서버 부하를 줄이고 안정성 높임
2.  **이중 안전장치**: `yf.Tickers`(대량 조회)로 먼저 시도하고, 정보가 없으면 `yf.Ticker`(개별 조회)로 다시 확인하여 데이터 수집 성공률을 극대화함
3.  **속도 조절(Sleep)**: 너무 빠른 요청으로 인한 IP 차단을 방지하기 위해 0.05초의 지연 시간을 둡니다.

In [ ]:
# 4. 정보 수집 실행
company_info = get_company_info(others_companies)    # pyright: ignore[reportUndefinedVariable]

# 결과 확인
info_df = pd.DataFrame.from_dict(company_info, orient='index')
print(info_df.head())

Fetching data from Yahoo Finance...


Processing Chunks: 0it [00:00, ?it/s]

Empty DataFrame
Columns: []
Index: []


In [7]:
# 5. 데이터 병합 및 업데이트
# 매핑 딕셔너리 생성
sector_map = info_df['Sector'].to_dict()
industry_map = info_df['Industry'].to_dict()

# 기존 Sector가 'Others'인 경우에만 업데이트
mask = df_daily['Sector'] == 'Others'
mapped_sectors = df_daily.loc[mask, 'Company'].map(sector_map)
df_daily.loc[mask, 'Sector'] = mapped_sectors.fillna('Others')

# Industry 컬럼은 새로 생성
df_daily['Industry'] = df_daily['Company'].map(industry_map).fillna('Unknown')

print("업데이트 후 섹터 분포:")
print(df_daily['Sector'].value_counts())

KeyError: 'Sector'

In [ ]:
# 6. 저장
df_daily.to_csv(DATA_DIR / 'stock_daily_master.csv', index=False)
